# Show Certificate Expiration Dates
This notebook will demonstrate the platform APIs for retrieving certificate expiration dates. 

## Setup

Set the `YB_PLATFORM_URL` and `YB_API_KEY` environment variables in a .env file before running this notebook.

Note: "verify=False" is used to disable SSL verification for the purposes of this demonstration, but you should use appropriate certificates in a production environment.


In [ ]:
import os
import requests
from pprint import pprint
from dotenv import load_dotenv
from datetime import datetime, timezone

load_dotenv()

yba_url = os.getenv("YBA_PLATFORM_URL")
yba_api_key = os.getenv("YBA_API_KEY")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{yba_api_key}"
}

session = requests.Session()
session.verify = False
session.headers = headers

pprint(yba_url)
pprint(yba_api_key)
pprint(headers)

## Get the customer ID

The customer ID is unique to the YBA platform and is required for the following Platform API calls. 

In [ ]:
route = f"{yba_url}/api/v1/session_info"
response = session.get(url=route).json()
customer_uuid = response["customerUUID"]

print(customer_uuid)

## List certificates

List a customer's certificates.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/5a40faca3973c-list-a-customer-s-certificates

In [ ]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/certificates"
response = session.get(url=route)
metrics = response.json()

today = datetime.now(timezone.utc)

for item in metrics:
    start_date = datetime.fromisoformat(item['startDateIso'].replace('Z', '+00:00'))
    expiry_date = datetime.fromisoformat(item['expiryDateIso'].replace('Z', '+00:00'))
    time_to_expiry = expiry_date - today
    
    print(f"{item['label']}\n"
          f"created: {start_date}\n"
          f"expiration: {expiry_date}\n"
          f"time to expiry: {time_to_expiry}\n")